In [1]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import concurrent.futures
from selenium.common.exceptions import StaleElementReferenceException


In [2]:
data = {
    "keywords": "jokowi",
    "since_time": "2023-10-16",
    "until_time": "2023-10-19"
}

In [3]:
def scrape_pagination(keywords, since_time, until_time):
    current_date = datetime.strptime(since_time, "%Y-%m-%d")
    until_date = datetime.strptime(until_time, "%Y-%m-%d")
    result_data = []  # Membuat list kosong untuk menyimpan data
    
    while current_date <= until_date:
        formatted_date = current_date.strftime("%d/%m/%Y")
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        url = f"https://www.detik.com/search/searchall?query={keywords}&siteid=2&sortby=time&fromdatex={formatted_date}&todatex={formatted_date}&page=1"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        cek = soup.find('span', {"class": "fl text"})
        
        if cek:
            hasil_text = cek.get_text()
            result = re.search(r'\d+', hasil_text)
            hasil_number = int(result.group(0))
            hasil_divided = hasil_number / 9
            page_index = round(hasil_divided)
            # print(formatted_date)
            result_data.append({
                'keywords':keywords,
                'tanggal_berita': formatted_date,
                'jumlah_index': page_index
            })
        else:
            print(f"No data found for {formatted_date}")
        current_date += timedelta(days=1)
    
    return result_data 

In [6]:
data_tanggal = scrape_pagination(data["keywords"], data["since_time"], data["until_time"])
print(data_tanggal)

[{'keywords': 'jokowi', 'tanggal_berita': '16/10/2023', 'jumlah_index': 8}, {'keywords': 'jokowi', 'tanggal_berita': '17/10/2023', 'jumlah_index': 11}, {'keywords': 'jokowi', 'tanggal_berita': '18/10/2023', 'jumlah_index': 11}, {'keywords': 'jokowi', 'tanggal_berita': '19/10/2023', 'jumlah_index': 8}]


In [7]:
def scrape_links(page_number, keywords, date, link_list):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://www.detik.com/search/searchall?query={keywords}&siteid=2&sortby=time&fromdatex={date}&todatex={date}&page={page_number}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article')

    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)

    print(f"Scraped {len(page_links)} links from page {page_number} cek tanggal {date}")
    
    # Extend the link_list with the links scraped on this page
    link_list.extend(page_links)

In [8]:
def scrape_all_links(data_tanggal):
    link_list = []  # Create an empty list to store all the links
    with ThreadPoolExecutor(max_workers=4) as executor:  # You can adjust the number of threads as needed
        for item in data_tanggal:
            keywords = item['keywords']
            date = item['tanggal_berita']
            max_pages = item['jumlah_index']
            for page_number in range(1, max_pages+1):  # Specify the range of pages you want to scrape
                executor.submit(scrape_links, page_number, keywords, date, link_list)
    
    return link_list

In [9]:
all_links = scrape_all_links(data_tanggal)
print(len(all_links))

Scraped 9 links from page 4 cek tanggal 16/10/2023
Scraped 9 links from page 1 cek tanggal 16/10/2023
Scraped 9 links from page 3 cek tanggal 16/10/2023
Scraped 9 links from page 2 cek tanggal 16/10/2023
Scraped 9 links from page 8 cek tanggal 16/10/2023
Scraped 9 links from page 7 cek tanggal 16/10/2023
Scraped 9 links from page 1 cek tanggal 17/10/2023
Scraped 9 links from page 5 cek tanggal 16/10/2023
Scraped 9 links from page 2 cek tanggal 17/10/2023
Scraped 9 links from page 3 cek tanggal 17/10/2023
Scraped 9 links from page 4 cek tanggal 17/10/2023
Scraped 9 links from page 5 cek tanggal 17/10/2023
Scraped 9 links from page 6 cek tanggal 17/10/2023
Scraped 9 links from page 7 cek tanggal 17/10/2023
Scraped 9 links from page 6 cek tanggal 16/10/2023
Scraped 9 links from page 8 cek tanggal 17/10/2023
Scraped 9 links from page 9 cek tanggal 17/10/2023
Scraped 4 links from page 11 cek tanggal 17/10/2023
Scraped 9 links from page 10 cek tanggal 17/10/2023
Scraped 9 links from page 2 c

In [14]:
def scrape_url(url, keywords,max_retries=3):
    retries = 0
    while retries < max_retries:
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
                }
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')

                    # Extract data from the web page
                    title_elem = soup.find('h1', {"class": "detail__title"})
                    title_text = title_elem.text.strip() if title_elem else "Title not found"

                    date_elem = soup.find('div', {"class": "detail__date"})
                    date_text = date_elem.text.strip() if date_elem else "Date not found"

                    body_elem = soup.find('div', {"class": "detail__body"})
                    if body_elem:
                        content_elem = body_elem.find_all('p')
                        content_text = "\n".join(p.text.strip() for p in content_elem)
                        content_text = content_text.replace('\n', ' ')
                    else:
                        content_text = "Content not found"

                    return {
                        'title': title_text,
                        'keywords': keywords,
                        'date': date_text,
                        'content': content_text,
                        'nama_berita': 'detik',
                        'link': url
                    }
                else:
                    print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error fetching URL '{url}': {e}")
            except Exception as e:
                print(f"Error processing URL '{url}': {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying {url} (Attempt {retries}/{max_retries})")
                time.sleep(5)  # You can adjust the delay as needed
    return None         

In [15]:
# Create a list to store the results
results = []

# Define a function to scrape URLs and store results
def scrape_and_store_result(url):
    result = scrape_url(url, data['keywords'])
    if result:
        results.append(result)

# Create a ThreadPoolExecutor to run the scraping function concurrently
max_threads = 5  # You can adjust the number of threads as needed
with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
    executor.map(scrape_and_store_result, all_links)

# Print or process the results as needed
for result in results:
    print(result)

{'title': 'Basarah Ungkap Suasana di PDIP soal Arah Politik 2024 Jokowi', 'keywords': 'jokowi', 'date': 'Senin, 16 Okt 2023 17:55 WIB', 'content': 'Arah politik Presiden sekaligus kader PDIP Joko Widodo (Jokowi) untuk Pilpres 2024 masih menjadi spekulasi. Ketua DPP PDIP Ahmad Basarah mengungkapkan suasana PDIP terkait sikap Jokowi di Pilpres 2024. "Beberapa minggu terakhir ini banyak spekulasi yang berkembang di alam pikir dan suasana kebatinan, bukan hanya PDIP, tapi juga rakyat Indonesia keseluruhan mengenai bagaimana sebenarnya sikap arah dan keputusan Pak Jokowi terhadap pilpres ini," kata Basarah dilansir detikJatim, Senin (16/10/2023). Basarah lalu menceritakan soal kedekatan dirinya dengan Jokowi, terutama saat ia mendampingi Jokowi kampanye pada tahapan Pilpres 2014. Basarah yakin Jokowi saat ini sedang berusaha menjalankan politik persatuan, seperti yang dilakukan Presiden Sukarno dan Ketua Umum PDIP Megawati Soekarnoputri. ADVERTISEMENT SCROLL TO CONTINUE WITH CONTENT "Dia me

In [12]:
results

[{'title': 'Basarah Ungkap Suasana di PDIP soal Arah Politik 2024 Jokowi',
  'keywords': 'jokowi',
  'date': 'Senin, 16 Okt 2023 17:55 WIB',
  'content': 'Arah politik Presiden sekaligus kader PDIP Joko Widodo (Jokowi) untuk Pilpres 2024 masih menjadi spekulasi. Ketua DPP PDIP Ahmad Basarah mengungkapkan suasana PDIP terkait sikap Jokowi di Pilpres 2024. "Beberapa minggu terakhir ini banyak spekulasi yang berkembang di alam pikir dan suasana kebatinan, bukan hanya PDIP, tapi juga rakyat Indonesia keseluruhan mengenai bagaimana sebenarnya sikap arah dan keputusan Pak Jokowi terhadap pilpres ini," kata Basarah dilansir detikJatim, Senin (16/10/2023). Basarah lalu menceritakan soal kedekatan dirinya dengan Jokowi, terutama saat ia mendampingi Jokowi kampanye pada tahapan Pilpres 2014. Basarah yakin Jokowi saat ini sedang berusaha menjalankan politik persatuan, seperti yang dilakukan Presiden Sukarno dan Ketua Umum PDIP Megawati Soekarnoputri. ADVERTISEMENT SCROLL TO CONTINUE WITH CONTENT 

In [13]:
df = pd.DataFrame(results)
print('hasil scrapping',len(results))
df

hasil scrapping 337


,title,keywords,date,content,nama_berita,link
0,Basarah Ungkap Suasana di PDIP soal Arah Polit...,jokowi,"Senin, 16 Okt 2023 17:55 WIB",Arah politik Presiden sekaligus kader PDIP Jok...,news.detik,https://news.detik.com/pemilu/d-6985398/basara...
1,Pakar Politik UGM Sebut Putusan MK Jadi Langka...,jokowi,"Senin, 16 Okt 2023 18:47 WIB",Pakar politik UGM Wawan Mas'udi mengkritik put...,detik,https://www.detik.com/jogja/berita/d-6985571/p...
2,Projo Ungkap Hampir Pasti Gibran Cawapres Prabowo,jokowi,"Senin, 16 Okt 2023 19:11 WIB",Relawan Pro-Jokowi (Projo) mengungkapkan hampi...,detik,https://www.detik.com/bali/nusra/d-6985605/pro...
3,"Gugatannya Dikabulkan MK, Mahasiswa Unsa Sebut...",jokowi,"Senin, 16 Okt 2023 18:49 WIB",Mahkamah Konstitusi (MK) mengabulkan uji mater...,detik,https://www.detik.com/jateng/berita/d-6985573/...
4,Apa Itu AALCO? Forum ke-61 Akan Diadakan Oktob...,jokowi,"Senin, 16 Okt 2023 17:36 WIB",Indonesia terpilih menjadi tuan rumah diseleng...,news.detik,https://news.detik.com/berita/d-6985364/apa-it...
...,...,...,...,...,...,...
332,Andre Rosiade Pastikan Prabowo dan Cawapresnya...,jokowi,"Kamis, 19 Okt 2023 14:28 WIB","Juru Bicara Koalisi Indonesia Maju (KIM), Andr...",detik,https://www.detik.com/jateng/berita/d-6991022/...
333,Andre Rosiade: Prabowo dan Cawapresnya Pastika...,jokowi,"Kamis, 19 Okt 2023 13:30 WIB",Ketua umum-ketua umum Koalisi Indonesia Maju (...,news.detik,https://news.detik.com/pemilu/d-6990864/andre-...
334,'Pesan Tuhan' di Balik Kemeja Mahfud di 2019 K...,jokowi,"Kamis, 19 Okt 2023 13:29 WIB",Bacawapres Mahfud Md mengenakan kemeja putih y...,news.detik,https://news.detik.com/pemilu/d-6990853/pesan-...
335,"Kode Cawapres Prabowo Muda-Berpengalaman, Gibr...",jokowi,"Kamis, 19 Okt 2023 14:01 WIB",Partai Gerindra seolah sudah memberikan kode m...,detik,https://www.detik.com/jateng/berita/d-6990941/...
